In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

In [ ]:
df = pd.read_csv('data.csv')

# EDA


#### Objetivos: 

- Entender melhor o dataset
- Determinar quais features são mais relevantes e podem ser utilizadas na vida real
- Supomos que essa amostra é representativa
- 299 amostras

In [ ]:
df.columns

In [ ]:
df.isna().sum()

In [ ]:
normal_levels = {
    'creatinine_phosphokinase': {
        'min': 32,
        'max': 294
    },
    'ejection_fraction': {
        'min': 50,
        'max': 70
    },
    'platelets': {
        'min': 150000,
        'max': 450000
    },
    'serum_creatinine': {
        'min': 0.6,
        'max': 1.2
    },
    'serum_sodium': {
        'min': 135,
        'max': 145
    },
    'age': {
        'min': 40,
        'max': 100
    }
}

In [ ]:
df.info()

In [ ]:
boolean_variables = ['anaemia', 'diabetes', 'high_blood_pressure', 'smoking','sex']

In [ ]:
continuous_variables = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium']

In [ ]:
for var in normal_levels.keys():
    fig, axes = plt.subplots(1, 2, figsize=(20, 6))

    # Histograma para casos de morte e não morte
    sns.histplot(df[df['DEATH_EVENT'] == 1][var], bins=20, kde=True, color='red', label='Morte', alpha=0.6, ax=axes[0])
    sns.histplot(df[df['DEATH_EVENT'] == 0][var], bins=20, kde=True, color='blue', label='Não Morte', alpha=0.6, ax=axes[0])

    # Adicionar linhas verticais para os valores mínimos e máximos
    axes[0].axvline(normal_levels[var]['min'], color='green', linestyle='--', label='Min')
    axes[0].axvline(normal_levels[var]['max'], color='orange', linestyle='--', label='Max')

    axes[0].set_title(f'Distribuição da {var} - Casos de Morte e Não Morte')
    axes[0].set_xlabel(var)
    axes[0].set_ylabel('Frequência')
    axes[0].legend()

    # Calcular a porcentagem de mortos de acordo com a variável
    age_death_percent = df.groupby(var)['DEATH_EVENT'].mean() * 100

    # Calcular a média móvel
    window_size = 3
    #window_size = 10
    age_death_percent_moving_avg = age_death_percent.rolling(window=window_size).mean()

    # Plotar a porcentagem de mortos de acordo com a idade
    sns.scatterplot(x=age_death_percent.index, y=age_death_percent.values, label='Porcentagem de Mortes', color='blue', ax=axes[1])
    sns.lineplot(x=age_death_percent_moving_avg.index, y=age_death_percent_moving_avg.values, label='Média Móvel', color='red', ax=axes[1])

    # Adicionar linhas verticais para os valores mínimos e máximos
    axes[1].axvline(normal_levels[var]['min'], color='green', linestyle='--', label='Min')
    axes[1].axvline(normal_levels[var]['max'], color='orange', linestyle='--', label='Max')

    axes[1].set_title(f'Porcentagem de Mortes de Acordo com {var}')
    axes[1].set_xlabel(var)
    axes[1].set_ylabel('Porcentagem de Mortes (%)')
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.show()

In [ ]:

for variable in boolean_variables:
    fig, axes = plt.subplots(1, 2, figsize=(20, 6))

    # Create a contingency table
    contingency_table = pd.crosstab(df[variable], df['DEATH_EVENT'])

    # Perform Chi-Square Test
    chi2, p, dof, expected = chi2_contingency(contingency_table)

    # print(f"Chi-Square Test Statistic: {chi2}")
    # print(f"P-Value: {p}")

    # Visualize the relationship
    sns.countplot(x=variable, hue='DEATH_EVENT', data=df, ax=axes[0])
    axes[0].set_xlabel(variable)
    axes[0].set_ylabel('Count')
    axes[0].set_title(f'Relationship between {variable} and Death')
    axes[0].legend(title='Death', loc='upper right', labels=['No', 'Yes'])

    # Calculate the percentage of deaths for each variable
    death_percent = df.groupby(variable)['DEATH_EVENT'].mean() * 100
    #print(f"Probability of death for {variable}:\n{death_percent}")

    # Plot the probability of death
    sns.barplot(x=death_percent.index, y=death_percent.values, palette='viridis', ax=axes[1])
    axes[1].set_xlabel(variable)
    axes[1].set_ylabel('Probability of Death (%)')
    axes[1].set_title(f'Probability of Death by {variable}')
    axes[1].set_ylim(0, 100)

    plt.tight_layout()
    plt.show()

1. Alta disponibilidade, ideal para produtos a serem utilizados na vida real
2. Feature time será descartada, pois isso não está disponível antes do paciente morrer
3. 


## Model Selection

- Escolha da metrica: é mais grave dizer que uma pessoa não é doente quando ela de fato é, que dizer que ela é doente e ela não ser
- Recall e Confusion Matrix
- Cuidado para aplicar mesmo pipeline
- Distribuicao parecida do y_train e do y_test

In [ ]:
features = ['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
            'ejection_fraction', 'high_blood_pressure', 'platelets',
            'serum_creatinine', 'serum_sodium', 'sex', 'smoking']
target = 'DEATH_EVENT'

X = df[features]
y = df[target]

# Dividir o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)
#

clf = RandomForestClassifier(
    n_estimators=100,
    class_weight={0: 1, 1: 3},  # Peso maior para a classe "1" (doentes)
    random_state=42
)

In [ ]:
pipeline = make_pipeline( RandomForestClassifier(class_weight={0: 1, 1: 3}, random_state=42))

# Definir os parâmetros para o Grid Search
param_grid = {
    'randomforestclassifier__n_estimators': [50, 100, 200],
    'randomforestclassifier__max_depth': [None, 10, 20, 30],
    'randomforestclassifier__min_samples_split': [2, 5, 10],
    'randomforestclassifier__min_samples_leaf': [1, 2, 4],
    'randomforestclassifier__class_weight': [{0: 1, 1: 1}, {0: 1, 1: 2}, {0: 1, 1: 3}]
}

# Realizar o Grid Search com validação cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='recall', n_jobs=-1)
grid_search.fit(X_train, y_train)

In [ ]:
# Exibir os melhores parâmetros e a melhor pontuação
print(f'Best parameters: {grid_search.best_params_}')
print(f'Best cross-validation recall: {grid_search.best_score_}')

# Avaliar o modelo no conjunto de teste
test_score = grid_search.best_estimator_.score(X_test, y_test)
print(f'Recall on test set: {test_score}')

In [ ]:
grid_search.best_params_

In [ ]:

# Obter o melhor modelo do grid search
best_model = grid_search.best_estimator_.named_steps['randomforestclassifier']
importances = best_model.feature_importances_
indices = np.argsort(importances)[::-1]

# Plotar as importâncias das features
plt.figure(figsize=(12, 8))
plt.title('Feature Importances')
plt.barh(range(X_train.shape[1]), importances[indices], align='center')
plt.yticks(range(X_train.shape[1]), [features[i] for i in indices])
plt.xlabel('Importance')
plt.ylabel('Features')
plt.gca().invert_yaxis()  # Inverter o eixo y para que a feature mais importante fique no topo
plt.tight_layout()
plt.show()

In [ ]:
y_pred = grid_search.best_estimator_.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No Death', 'Death'])
disp.plot(cmap='inferno')
plt.title('Confusion Matrix ')
plt.show()


In [ ]:
y_pred = grid_search.best_estimator_.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm_percentage = cm.astype('float') / cm.sum(axis=0)[:, np.newaxis] * 100

disp = ConfusionMatrixDisplay(confusion_matrix=cm_percentage, display_labels=['No Death', 'Death'])
disp.plot(cmap='inferno')
plt.title('Confusion Matrix (Percentage)')
plt.show()


In [ ]:
# Salvar o melhor modelo como um arquivo .joblib
joblib_file = "pipeline_model.joblib"
joblib.dump(grid_search.best_estimator_, joblib_file)
print(f'Best pipeline saved as {joblib_file}')